In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
import ee
import geemap
from utils import ee_utils, date_utils, plot_config
from utils.ee_utils import back_to_float

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from typing import Dict, List, Tuple, Union

import warnings

warnings.filterwarnings("ignore")

In [ ]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

## Validating ET Blue Maps with Walter Koch's Estimates

This notebook follows a three-step process:

1. **Create Yearly ET Blue Maps (2018-2022):**  
   - Aggregate dekadal ET Blue maps into monthly maps.
   - Filter the maps to include only the growing season months (April through October).
   - Sum the monthly ET Blue maps to produce yearly ET Blue maps.

2. **Calculate Yearly ET Blue Values for Potentially Irrigated Fields:**  
   - Compute ET blue in m3/ha/yr for each crop field for each year.

3. **Compare ET Blue Values with Walter Koch's Estimates:**
   - Make pretty plots

## Aggregate dekadal ET Blue maps into monthly maps

In [12]:
ET_blue_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_wapor_monthly_2018-2023"
)


ET_blue_collection = ET_blue_collection.map(lambda img: back_to_float(img, 100))

In [13]:
date_utils.print_collection_dates(ET_blue_collection)

Dates of images in the collection:
2018-01-01
2018-02-01
2018-03-01
2018-04-01
2018-05-01
2018-06-01
2018-07-01
2018-08-01
2018-09-01
2018-10-01
2018-11-01
2018-12-01
2019-01-01
2019-02-01
2019-03-01
2019-04-01
2019-05-01
2019-06-01
2019-07-01
2019-08-01
2019-09-01
2019-10-01
2019-11-01
2019-12-01
2020-01-01
2020-02-01
2020-03-01
2020-04-01
2020-05-01
2020-06-01
2020-07-01
2020-08-01
2020-09-01
2020-10-01
2020-11-01
2020-12-01
2021-01-01
2021-02-01
2021-03-01
2021-04-01
2021-05-01
2021-06-01
2021-07-01
2021-08-01
2021-09-01
2021-10-01
2021-11-01
2021-12-01
2022-01-01
2022-02-01
2022-03-01
2022-04-01
2022-05-01
2022-06-01
2022-07-01
2022-08-01
2022-09-01
2022-10-01
2022-11-01
2022-12-01
2023-01-01
2023-02-01
2023-03-01
2023-04-01
2023-05-01
2023-06-01
2023-07-01
2023-08-01
2023-09-01
2023-10-01
2023-11-01
2023-12-01


In [ ]:
ee_utils.print_value_ranges(ET_blue_collection)

In [19]:
ET_blue_collection_list = ET_blue_collection.toList(ET_blue_collection.size())

Map = geemap.Map()

image = ee.Image(ET_blue_collection_list.get(20))

vis_params = {
    "bands": ["ET_blue"],
    "min": 0,
    "max": 10,
    "palette": ["white", "blue", "green", "yellow", "red"],
}

Map.addLayer(image, vis_params, "ET_blue")
Map.center_object(image, 10)
Map.add_colorbar(vis_params, label="ET_blue", orientation="horizontal")
Map

Map(center=[47.536764711902414, 9.088557224439409], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
ET_blue_collection_monthly = ee_utils.aggregate_to_monthly(
    ET_blue_collection, bands=["ET_blue", "ET_blue_m3"]
)

## Filter the maps to include only the growing season months (April through October).

In [ ]:
ET_blue_collection_monthly_growing_season = ET_blue_collection_monthly.filter(ee.Filter.calendarRange(4, 10, "month"))

date_utils.print_collection_dates(ET_blue_collection_monthly_growing_season)

## Sum the monthly ET Blue maps to produce yearly ET Blue maps.

In [ ]:
def create_growing_season_yearly_sums(
    collection: ee.ImageCollection,
) -> ee.ImageCollection:
    """
    Create yearly sums from a collection of monthly images for the growing season (April to October).

    Args:
        collection (ee.ImageCollection): Input collection with monthly images for the growing season.

    Returns:
        ee.ImageCollection: Collection with yearly sum images for the growing season.
    """

    def sum_growing_season(year):
        start_date = ee.Date.fromYMD(year, 4, 1)  # April 1st
        end_date = ee.Date.fromYMD(year, 11, 1)  # November 1st (exclusive)
        yearly_sum = collection.filterDate(start_date, end_date).sum()
        return yearly_sum.set("year", year).set(
            "system:time_start", start_date.millis()
        )

    # Get the range of years in the collection
    start_year = ee.Date(collection.first().get("system:time_start")).get("year")
    end_year = ee.Date(
        collection.sort("system:time_start", False).first().get("system:time_start")
    ).get("year")

    # Create a list of years
    years = ee.List.sequence(start_year, end_year)

    # Map over the years to create yearly sums
    yearly_sums = ee.ImageCollection.fromImages(years.map(sum_growing_season))

    return yearly_sums


ET_blue_collection_yearly = create_growing_season_yearly_sums(
    ET_blue_collection_monthly_growing_season
)

In [ ]:
ET_blue_collection_yearly.first().bandNames().getInfo()

## Compute ET blue in m3/ha/yr for each crop field for each year and export

In [ ]:
def calculate_et_blue_per_field(
    et_blue_image: ee.Image,
    crop_fields: ee.FeatureCollection,
    scale: Union[int, float] = 10,
) -> ee.FeatureCollection:
    """
    Calculate ET_blue in m3/ha/yr for each crop field, handling null values and edge cases.

    Args:
        et_blue_image (ee.Image): Image containing ET_blue estimates in m3/yr.
        crop_fields (ee.FeatureCollection): Collection of crop field features.
        scale (Union[int, float], optional): Scale in meters for computations. Defaults to 10.

    Returns:
        ee.FeatureCollection: Updated crop fields with ET_blue_m3_ha_yr property.
    """

    def calculate_et_blue(feature: ee.Feature) -> ee.Feature:
        geometry = feature.geometry()

        # Calculate mode of ET blue pixels within the field
        mode_stats = et_blue_image.reduceRegion(
            reducer=ee.Reducer.mode(), geometry=geometry, scale=scale, maxPixels=1e9
        ).get("ET_blue_m3")

        # Calculate area in hectares - do this outside the conditional to avoid repeated computation
        area_ha = ee.Number(geometry.area()).divide(10000)

        def compute_when_irrigated():
            # Calculate total ET blue volume
            et_blue_sum = et_blue_image.reduceRegion(
                reducer=ee.Reducer.sum(), geometry=geometry, scale=scale, maxPixels=1e9
            ).get("ET_blue_m3")

            # Handle potential null values in sum calculation
            return ee.Number(
                ee.Algorithms.If(
                    ee.Algorithms.IsEqual(et_blue_sum, None),
                    0,
                    ee.Number(et_blue_sum).divide(area_ha).round(),
                )
            )

        # Use nested If statements to handle null values
        et_blue_m3_ha_yr = ee.Number(
            ee.Algorithms.If(
                ee.Algorithms.IsEqual(mode_stats, None),
                0,  # If mode_stats is null, return 0
                ee.Algorithms.If(
                    ee.Number(mode_stats).eq(0),
                    0,  # If mode is 0, return 0
                    compute_when_irrigated(),  # Otherwise calculate the value
                ),
            )
        )

        return feature.set("ET_blue_m3_ha_yr", et_blue_m3_ha_yr)

    return crop_fields.map(calculate_et_blue)

In [ ]:
def export_feature_collection(collection: ee.FeatureCollection, task_name: str, asset_id: str): 
    """
    Export the feature collection to an Earth Engine asset.

    Args:
        collection: The feature collection to export.
        year: The year of the feature collection.
        task_name: The name of the export task.
        asset_id: The asset ID to export to.
    """
    task = ee.batch.Export.table.toAsset(
        collection=collection,
        description=task_name,
        assetId=asset_id,
    )
    task.start()

In [ ]:
years = range(2018, 2023)

not_irrigated_crops = get_crops_to_exclude()
exclude_filter, _ = create_crop_filters(not_irrigated_crops, not_irrigated_crops)

ET_blue_collection_yearly_list = ET_blue_collection_yearly.toList(
    ET_blue_collection_yearly.size()
)

for year in years:
    index = year - 2018
    nutzung_collection_year = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
    )

    potentially_rainfed, _ = filter_crops(nutzung_collection_year, exclude_filter, _)

    et_blue_image = ee.Image(ET_blue_collection_yearly_list.get(index))

    crop_with_et_blue = calculate_et_blue_per_field(et_blue_image, potentially_rainfed)

    task_name = f"ET_blue_m3_ha_yr_per_field_{year}"
    asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_m3_ha_yr_per_field_2018-2022/{task_name}"

    export_feature_collection(crop_with_et_blue, task_name, asset_id)
    print(f"Exporting {task_name} to {asset_id}")

In [ ]:
# crop_with_et_blue.first().getInfo()

In [ ]:
palette = ["blue", "green", "yellow", "red"]

# Define visualization parameters
visualization = {
    "min": 0,
    "max": 0.1,  # Adjust based on your maximum irrigation volume
    "palette": palette,
}

# Convert fields_w_estimates to an image for gradient visualization
fields = crop_with_et_blue.reduceToImage(
    properties=["ET_blue_m3_ha_yr"], reducer=ee.Reducer.first()
)

# Create a map centered on the area of interest
Map = geemap.Map(center=[47.63915833792603, 8.77542613019931], zoom=12)

# Add the gradient visualization layer to the map
Map.addLayer(fields, visualization, "ET_blue_m3_ha_yr")

Map

In [ ]:
# # Print feature with id =000000000000000048c7 in the nutzung_collection_year
# feature = crop_with_et_blue.filter(ee.Filter.eq("system:index", "000000000000000048c7")).first()
# feature.getInfo()

In [ ]:
# nutzung_collection_year = ee.FeatureCollection(
#     f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{2020}_area"
# )


# potentially_rainfed, _ = filter_crops(nutzung_collection_year, exclude_filter, _)

# et_blue_image = ee.Image(ET_blue_collection_yearly_list.get(2))

# crop_with_et_blue = calculate_et_blue_per_field(et_blue_image, potentially_rainfed)

## Comparing to Walter Koch's estimates

In [ ]:
irrigation_estimate = {
    "Einjährige Freilandgemüse, ohne Konservengemüse": [200, 1000],
    "Kartoffeln": [200, 1000],
    "Freiland-Konservengemüse": [200, 600],
}

IRRIGATION_EFFICIENCY = 0.5

In [ ]:
def filter_dataframe(df: pd.DataFrame, crop_types: List[str]) -> pd.DataFrame:
    """
    Filter the DataFrame to include only the specified crop types.

    Args:
        df (pd.DataFrame): Input DataFrame with 'ET_blue_m3_ha_yr' and 'nutzung' columns.
        crop_types (List[str]): List of crop types to include.

    Returns:
        pd.DataFrame: Filtered DataFrame.
    """
    return df[df["nutzung"].isin(crop_types)]


def plot_histogram_comparison(
    calculated_et_blue: pd.DataFrame,
    farmer_estimates: Dict[str, Tuple[int, int]],
    output_destination: str,
) -> None:
    """
    Create a histogram comparison of calculated ET blue vs farmer estimates.

    Args:
        calculated_et_blue (pd.DataFrame): DataFrame with 'ET_blue_m3_ha_yr' and 'nutzung' columns.
        farmer_estimates (Dict[str, Tuple[int, int]]): Dictionary of farmer estimates for each crop type.
        output_destination (str): Output destination for the plot.
    """
    plot_config.set_plot_style()

    crop_types = list(farmer_estimates.keys())
    num_crops = len(crop_types)

    fig, axs = plt.subplots(1, num_crops, figsize=(6 * num_crops, 6))

    for i, crop in enumerate(crop_types):
        crop_data = calculated_et_blue[calculated_et_blue["nutzung"] == crop][
            "ET_blue_m3_ha_yr"
        ]

        sns.histplot(crop_data, kde=True, ax=axs[i])
        axs[i].axvline(
            farmer_estimates[crop][0],
            color="r",
            linestyle="--",
            label="Farmer's estimate range",
        )
        axs[i].axvline(farmer_estimates[crop][1], color="r", linestyle="--")

        axs[i].set_xlabel("ET Blue (m³/ha/yr)")
        axs[i].set_ylabel("Frequency")
        axs[i].set_title(f"{crop}")

        # Add summary statistics
        mean_value = crop_data.mean()
        median_value = crop_data.median()
        axs[i].axvline(
            mean_value, color="g", linestyle="-", label=f"Average: {mean_value:.0f}"
        )

        axs[i].legend(loc="upper center", bbox_to_anchor=(0.5, -0.15), ncol=2)

    plt.tight_layout()

    # plt.savefig(output_destination, bbox_inches="tight", dpi=300)
    plt.subplots_adjust(bottom=0.2)

In [ ]:
for year in range(2018, 2023):
    ET_blue_per_field = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_m3_ha_yr_per_field_2018-2022/ET_blue_m3_ha_yr_per_field_{year}"
    )

    ET_blue_per_field_year_df = geemap.ee_to_df(ET_blue_per_field)
    ET_blue_per_field_year_df = ET_blue_per_field_year_df[
        ["ET_blue_m3_ha_yr", "nutzung"]
    ]

    ET_blue_per_field_year_df["ET_blue_m3_ha_yr"] = (
        ET_blue_per_field_year_df["ET_blue_m3_ha_yr"] / IRRIGATION_EFFICIENCY
    )

    # Filter the DataFrame
    filtered_df = filter_dataframe(
        ET_blue_per_field_year_df, list(irrigation_estimate.keys())
    )
    filtered_df = filtered_df[filtered_df["ET_blue_m3_ha_yr"] > 0]

    # Create the plot
    plot_histogram_comparison(
        filtered_df,
        irrigation_estimate,
        output_destination=f"/Users/cooper/Desktop/Hydrosolutions/ETblue-estimation /images/ET_blue_validation/ET_blue_histogram_comparison_{year}.png",
    )

In [ ]:
Map = geemap.Map()

irrigation_image = ET_blue_per_field.reduceToImage(
    properties=["ET_blue_m3_ha_yr"], reducer=ee.Reducer.first()
)

et_blue_postprocessed = ee.Image(ET_blue_collection_yearly.first())

vis_params = {
    "min": 0,
    "max": 300,
    "palette": ["white", "blue", "green", "yellow", "red"],
}

# vis_params_et_blue = {
#     "bands": ["ET_blue_m3"],
#     "min": 0,
#     "max": 10,
#     "palette": ["white", "blue", "green", "yellow", "red"],
# }

Map.addLayer(irrigation_image, vis_params, "ET_blue_m3_ha_yr")
# Map.addLayer(et_blue_postprocessed, vis_params_et_blue, "ET_blue_m3")

Map.add_colorbar(vis_params, label="ET_blue_m3_ha_yr", orientation="horizontal")
# Map.add_colorbar(vis_params_et_blue, label="ET_blue_m3", orientation="horizontal")

Map

## Field level estimates for 2022

In [ ]:
# fields_w_estimates = ee.FeatureCollection("projects/thurgau-irrigation/assets/ZH_Nutzungsflaechen_2/2022_with_irrigation_estimates")

In [ ]:
palette = ["blue", "green", "yellow", "red"]

# Define visualization parameters
visualization = {
    "min": 0,
    "max": 1000,  # Adjust based on your maximum irrigation volume
    "palette": palette,
}

# Convert fields_w_estimates to an image for gradient visualization
irrigation_image = fields_w_estimates.reduceToImage(
    properties=["estimated_irrigated_volume"], reducer=ee.Reducer.first()
)

# Create a map centered on the area of interest
Map = geemap.Map(center=[47.63915833792603, 8.77542613019931], zoom=12)

# Add the gradient visualization layer to the map
Map.addLayer(irrigation_image, visualization, "Irrigation Volume (Gradient)")
Map.addLayer(fields_w_estimates, {"color": "black"}, "Fields")

Map